In [1]:
# making changes to tempdata

In [2]:
import numpy as np
import json
import sys
import pandas as pd
import os
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
from os import listdir
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings("ignore")

In [3]:
sys.path.insert(0, 'src')
from helper import *
from eda import *
from train import *

### etl (new readfilerun and gen)

In [4]:
def readfilerun(run_, output_dir): # test_tempdata, tempdata
    outdir = os.path.join(os.getcwd() , run_, output_dir)
    names = listdir(run_) # all filenames in data
    daneruns = [x for x in names if not 'losslog' in x]
    daneruns = [ filename for filename in daneruns if filename.endswith('.csv') ]
    
    # daneruns = ['data/20220116T055105_20-100-true-20-100-iperf.csv', 'data/20220116T055942_20-250-true-20-250-iperf.csv']
    losslogs = [x for x in names if 'losslog' in x]
    #print(losslogs)
    for count, run in enumerate(daneruns):
        print(count)
        # print(run) #for debug
        #curr_losslog = losslogs[count] # 
       
        print("run: ", run)
        run_labels = run.split('_')[1].split('.')[0].split('-')
        #print("run_labels:", run_labels)
        temp_label_str = '-'.join(run_labels[:-1]) 
        #print(temp_label_str)
        
        
        losslog = f'{run_}/losslog-{temp_label_str}.csv' #losslog filename str

        print(losslog)
        run_df = pd.read_csv(f'{run_}/{run}')
        losslog_df = pd.read_csv(losslog, header=None).rename(
            columns={0:'event', 1:'drop_unix', 2:'IP1', 3:'Port1', 4:'IP2', 5:'Port2', 6:'Proto'})
        losslog_df['Time'] = losslog_df['drop_unix'].astype(int)
        losslog_df = losslog_df.groupby(['Time', 'IP1', 'Port1', 'IP2', 'Port2', 'Proto']).agg(lambda x: ';'.join(x.astype(str))).reset_index()
        
        df = pd.merge(run_df, losslog_df, on=['Time', 'IP1', 'Port1', 'IP2', 'Port2', 'Proto'], how="left") # merge on fivetuple key
        # df.fillna(inplace=True, value=-1) #TODO plan implementation of dealing with null values
        # df['event'] =  # TODO change to 3 different profiles: no drop, drop, and switch event
        df = df[df['Proto'] == df['Proto'].mode()[0]] # selects relevant non ipv6 int(connection
        
        ## adding labels
        df['latency'] = int(run_labels[0])
        df['loss'] = int(run_labels[1])
        df['later_latency'] = int(run_labels[3])
        df['later_loss'] = int(run_labels[4])
        df['deterministic'] = bool(run_labels[2])
        
        #TODO future implementation of boolean flag for when the switch happens so we know when to use later lat/loss
        
        df.to_csv(f'{outdir}/labeled-from-{run_}_{temp_label_str}.csv') # save to temporary output directory: just merging takes a bit

In [5]:
def list_to_csv(lst, filepth):
    '''takes list of pandas dataframes with similar column structure and outputs them to a single folder'''
    lst[0].to_csv(filepth, index=False)
    for i in range(1, len(lst)):
        lst[i].to_csv(filepth, index=False, header=False, mode='a')
    return

In [6]:
def subset_helper(cond, window_start):
    unseen = ""
    if cond == 'seen': 
        print("subset of seen data")
    if cond == 'unseen': 
        print("subset of un seen data")
        unseen = "unseen"
    
    window = 8 # 8 second window
    window_end = window_start + window
    
    tempdatafiles = listdir('data/tempdata')
    fnames = [ filename for filename in tempdatafiles if filename.endswith(".csv" ) ]
    
    datasubset = []
    for j in fnames:
        fileloc = os.path.join(os.getcwd(), 'data', "tempdata", j)
        print(fileloc)
        
        scaled = time(pd.read_csv(fileloc))
        scaled['total_pkts'] = scaled['1->2Pkts'] + scaled['2->1Pkts'] # combining packets
        scaled['total_bytes'] = scaled['1->2Bytes'] + scaled['2->1Bytes'] # combining bytes
        scaled_subset = scaled.iloc[window_start: window_end]
        datasubset.append(scaled_subset)
        
    path =  os.path.join(os.getcwd() , "outputs")
    outname = unseen +  "combined_subset" + str(window_start) + ".csv"
    print(outname)
    list_to_csv( datasubset, os.path.join(path, outname))


In [7]:
def gen(cond , subset):
    unseen = ""
    if cond == 'seen': 
        print("transforming seen data")
    if cond == 'unseen': 
        print("transforming un seen data")
        unseen = "unseen"
    tempdatafiles = listdir('data/tempdata')
    if unseen == 'unseen': 
        tempdatafiles = listdir('test/test_tempdata')
        
    fnames = [ filename for filename in tempdatafiles if filename.endswith(".csv" ) ]
    
    data, datasubset, transformed  = [], [], []
    for j in fnames:
        loc = os.path.join(os.getcwd(), 'data', "tempdata", j)
        if unseen == 'unseen': 
            loc = os.path.join(os.getcwd(), 'test', "test_tempdata", j)
            
        print(loc)
        df_cols = genfeat(pd.read_csv(loc))
        
        #data
        time_scaled = time(df_cols)
        data.append(time_scaled)
        
        #subset
#         df_mid = time_scaled.iloc[60:60+subset]
#         datasubset.append(df_mid)

        #transformed
        f_df = agg10(df_cols)
        transformed.append(f_df)
        
    # makes paths
    path = os.path.join(os.getcwd() , "outputs", "gen_temp")
    path2 = os.path.join(os.getcwd() , "outputs")
    
    list_to_csv(data, os.path.join(path2, unseen + "combined_all.csv"))
    #print('combined_all_finished', sep=' ')
#     list_to_csv(datasubset, os.path.join(path2, unseen + "combined_subset_6068.csv"))
    #print('combined_subset_finished', sep=' ')
    list_to_csv(transformed, os.path.join(path2, unseen +  "combined_transform.csv"))
    #print('transformed_finished', sep=' ')
        
    return None

In [8]:
def genviz(cond, fname, subset):
    unseen = ""
    if cond == 'seen': 
        print("transforming seen data")
    if cond == 'unseen': 
        print("transforming un seen data")
        unseen = "unseen"

    data, datasubset, transformed  = [], [], []
    #loc = os.path.join(os.getcwd(),"data", "tempdata", fname)
    loc =  os.path.join(os.getcwd(),"tempdata2", fname)
    
    t = pd.read_csv(loc)
    df_cols = genfeat(t)
    print(fname)
    run_labels = fname.split('_')[1].split('.')[0].split('-')
    print(run_labels)

    #data
    time_scaled = time(df_cols)
    data.append(time_scaled)

    #subset
    df_mid = time_scaled.iloc[:subset]
    datasubset.append(df_mid)

    #transformed
    f_df = agg10(df_cols)
    transformed.append(f_df)
    
    path = os.path.join(os.getcwd() , "outputs", "gen_temp")
    path2 = os.path.join(os.getcwd() , "outputs")
    label = run_labels[0] + '-' + run_labels[1] +  '__' + run_labels[3] + '-' + run_labels[4]
    temp_data = pd.concat(data , ignore_index=True)#.reset_index(drop = True)
    temp_data.to_csv(os.path.join(path2, unseen + "s_all_" + label + ".csv"), index = False)
    
    temp_subset = pd.concat(datasubset , ignore_index=True)
    temp_subset.to_csv(os.path.join(path2, unseen +  "s_subset_" + label + ".csv"), index = False)
    
    temp_t = pd.concat(transformed, ignore_index=True)  
    temp_t.to_csv(os.path.join(path2, unseen +  "s_transform_" + label + ".csv"), index = False)

#### eda

In [9]:
def main_eda(cond, lst, filen1, filen2, filen3):
    unseen = ''
    if cond =='unseen': 
        unseen = 'unseen'
    
    fpath1 = os.path.join(os.getcwd() , "outputs", unseen + filen1)
    df_1 = pd.read_csv(fpath1)
    fpath2 = os.path.join(os.getcwd() , "outputs", unseen + filen2)
    df_2 = pd.read_csv(fpath2)
    fpath3 = os.path.join(os.getcwd() , "outputs", unseen + filen3)
    df_3 = pd.read_csv(fpath3)
    
    plottogether(cond, lst, df_1, filen1.strip(".csv")) # trends over subset
    plottogether(cond, lst, df_3, filen3.strip(".csv")) # trends over entire data
    plotloss(cond, df_2)

    plot_correlation_matrix(cond, df_2) # correlation matrix
    plotlongest(df_3, cond, 600, 1400)
    # below makes rest of visualizations
    plotbytes(df_3, 600, 1400, 200, 200)
    #plot_detailed_bytes()

In [10]:
# overlaying model predictions and plotting detailed bytes

#### train

In [11]:
def test_feat(cond, df, cols, p, df_u): 
    unseen = ''
    if cond =='unseen': 
        unseen = 'unseen'
    # col is feauture comb
    # p is for loss or latency   1: loss  # 2 : latency
    X = df[cols]
    
    X2 = df_u[cols]

    if p == 1:  # flag found in test_mse
        y = df.loss
        y2 = df_u.loss
    if p == 2: 
        y = df.latency
        y2 = df_u.latency
        
    # randomly split into train and test sets, test set is 80% of data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

    if unseen == 'unseen': 
        X_test = X2
        y_test = y2
    
    clf = DecisionTreeRegressor()
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    acc1 = mean_squared_error(y_test, y_pred)
    
    clf2 = RandomForestRegressor(n_estimators=10)
    clf2 = clf2.fit(X_train,y_train)
    y_pred2 = clf2.predict(X_test)
    acc2= mean_squared_error(y_test, y_pred2)
    
    clf3 = ExtraTreesRegressor(n_estimators=10)
    clf3 = clf3.fit(X_train,y_train)
    y_pred3 = clf3.predict(X_test)
    acc3= mean_squared_error(y_test, y_pred3)
    
    pca = PCA() 
    X_transformed = pca.fit_transform(X_train) 
    cl = DecisionTreeRegressor() 
    cl.fit(X_transformed, y_train)
    newdata_transformed = pca.transform(X_test)
    y_pred4 = cl.predict(newdata_transformed)
    acc4 = mean_squared_error(y_test, y_pred4)
    
    clf_gbc = GradientBoostingRegressor(random_state=0)
    clf_gbc.fit(X_train, y_train)
    y_pred5 = clf_gbc.predict(X_test)
    acc5 = mean_squared_error(y_test, y_pred5) 
    return [acc1, acc2, acc3, acc4, acc5]


def test_mse(cond, all_comb1, all_comb2):
    unseen = ''
    if cond =='unseen': 
        unseen = 'unseen'
    filedir_unseen = os.path.join(os.getcwd(), "outputs", unseen + "combined_transform.csv")
    df_unseen = pd.read_csv(filedir_unseen)
    filedir = os.path.join(os.getcwd(), "outputs", "combined_transform.csv")
    df = pd.read_csv(filedir)
    
    all_comb1 = pd.Series(all_comb1).apply(lambda x: list(x))
    all_comb2 = pd.Series(all_comb2).apply(lambda x: list(x))
    
    dt = []
    rf = []
    et = []
    pca = []
    gbc = []
    for i in all_comb1:
        acc_loss = test_feat(cond, df, i, 1, df_unseen)
        dt.append(acc_loss[0])  
        rf.append(acc_loss[1])  
        et.append(acc_loss[2])   
        pca.append(acc_loss[3])   
        gbc.append(acc_loss[4])
        
    # optimze by adding a flag called losslat to avoid making two dataframes of results
    dt2 = []
    rf2 = []
    et2 = []
    pca2 = []
    gbc2 = []
    for i in all_comb2:
        # 1 = loss
        # 2 = latency
        acc_latency = test_feat(cond, df, i, 2, df_unseen)
        dt2.append(acc_latency[0])
        rf2.append(acc_latency[1])
        et2.append(acc_latency[2]) 
        pca2.append(acc_latency[3])
        gbc2.append(acc_latency[4])
        
    dict1 = pd.DataFrame({'feat': all_comb1, 'dt': dt, 'rf': rf, 'et': et, 'pca': pca, 'gbc': gbc})
    dict2 = pd.DataFrame({'feat2': all_comb2, 'dt2': dt2, 'rf2': rf2, 'et2': et2, 'pca2': pca2, 'gbc2': gbc2})
    
    path = os.path.join(os.getcwd() , "outputs")
    dict1.to_csv(os.path.join(path, unseen + "feat_df1.csv"), index = False)
    dict2.to_csv(os.path.join(path, unseen + "feat_df2.csv"), index = False)


def best_performance(cond):
    unseen = ''
    if cond == 'unseen': 
        unseen = 'unseen'
    #print("finding best loss performance")
    filedir1 = os.path.join(os.getcwd(), "outputs", unseen + "feat_df1.csv")
    df1 = pd.read_csv(filedir1)
    print( "\n")
    print("Loss Performance sorted from lowest to highest", "\n")
    print(df1.sort_values(by=['dt', 'rf', 'et', 'pca'], ascending = True)[:5], "\n")
    
    #print("finding best latency performance")
    filedir2 = os.path.join(os.getcwd(), "outputs", unseen + "feat_df2.csv")
    df2 = pd.read_csv(filedir2)
    print( "\n")
    print("Latency Performance sorted from lowest to highest", "\n")
    print(df2.sort_values(by=['dt2', 'rf2', 'et2', 'pca2'], ascending = True)[:5], "\n")


#### main

In [16]:
def main(targets):

    transform_config = json.load(open('config/transform.json'))
    columns = json.load(open('config/columns.json'))
    eda_config = json.load(open('config/eda.json'))
    all_config = json.load(open("config/all.json"))

    test_unseen = 'unseen'
    test_seen = 'seen'
    
    cond1 = True
    cond2 = False

    if 'data' in targets:
        """generating feat from unseen and seen data"""
        readfilerun('data', 'tempdata') 
        readfilerun('test', 'test_tempdata') 
        gen(test_seen, **transform_config)
        gen(test_unseen, **transform_config)

    if 'eda' in targets:  
#         readfilerun('data', 'tempdata') 
#         gen(test_seen, **transform_config)
        main_eda(test_seen, [200, 200], **eda_config)
        print("EDA saved to outputs/eda/ folder")

    if 'train' in targets:
        "trains tests in this target"
#         readfilerun('data', 'tempdata') 
#         gen(test_seen, **transform_config)
                
        comb1 = getAllCombinations(1)
        comb2 = getAllCombinations(2)
        
        print("Testing on seen data: ")
        test_mse(test_seen, comb1, comb2)
        best_performance(test_seen)
                        
    if "inference" in targets: 
        print('tba')
#         readfilerun('test', 'test_tempdata') 
#         gen(test_seen, **transform_config)
#         gen(test_unseen, **transform_config)
        
        comb1 = getAllCombinations(1)
        comb2 = getAllCombinations(2)
        
        print("Testing on unseen data: ")
        test_mse(test_unseen, comb1, comb2)
        best_performance(test_unseen)
            
    if "test" in targets: 
        """ runs all targets on sample data"""
        print('tba')
#         readfilerun('data', 'tempdata') 
#         gen(test_seen, **transform_config)
#         gen(test_unseen, **transform_config)
        
#         main_eda(test_seen, **eda_config)
#         print("EDA saved to outputs/eda/ folder")
        
#         comb1 = getAllCombinations(1)
#         comb2 = getAllCombinations(2)
        
#         print("Testing on seen data: ")
#         test_mse(test_seen, comb1, comb2)
#         best_performance(test_seen)
        
#         print("Testing on unseen data: ")
#         test_mse(test_unseen, comb1, comb2)
#         best_performance(test_unseen)
        
    if 'all' in targets: 
        print('tba')
        
# if __name__ == '__main__':

#     targets = sys.argv[1:]
#     main(targets)

In [13]:
targets = ['data', 'eda', 'train', 'inference', 'test', 'all']
# main(targets[0])

In [14]:
main(targets[0])

0
run:  20220117T055730_300-250-true-300-250-iperf.csv
data/losslog-300-250-true-300-250.csv
1
run:  20220117T002516_100-15000-true-100-15000-iperf.csv
data/losslog-100-15000-true-100-15000.csv
2
run:  20220118T022316_400-18000-true-400-18000-iperf.csv
data/losslog-400-18000-true-400-18000.csv
3
run:  20220117T044411_250-15000-true-250-15000-iperf.csv
data/losslog-250-15000-true-250-15000.csv
4
run:  20220118T011226_400-8000-true-400-8000-iperf.csv
data/losslog-400-8000-true-400-8000.csv
5
run:  20220117T015822_200-100-true-200-100-iperf.csv
data/losslog-200-100-true-200-100.csv
6
run:  20220117T045523_250-17000-true-250-17000-iperf.csv
data/losslog-250-17000-true-250-17000.csv
7
run:  20220117T002515_100-13000-true-100-13000-iperf.csv
data/losslog-100-13000-true-100-13000.csv
8
run:  20220117T235852_400-2000-true-400-2000-iperf.csv
data/losslog-400-2000-true-400-2000.csv
9
run:  20220117T085718_350-4000-true-350-4000-iperf.csv
data/losslog-350-4000-true-350-4000.csv
10
run:  20220118T

In [15]:
main(targets[2])

all combinations generated
all combinations generated
Testing on seen data: 


Loss Performance sorted from lowest to highest 

                                          feat   dt   rf   et  pca  \
0                              ['total_bytes']  0.0  0.0  0.0  0.0   
1               ['longest_seq', 'total_bytes']  0.0  0.0  0.0  0.0   
2                              ['longest_seq']  0.0  0.0  0.0  0.0   
3                 ['max_bytes', 'total_bytes']  0.0  0.0  0.0  0.0   
4  ['max_bytes', 'longest_seq', 'total_bytes']  0.0  0.0  0.0  0.0   

            gbc  
0  39480.454884  
1   6849.668570  
2  74037.977881  
3   7970.491391  
4   7178.137420   



Latency Performance sorted from lowest to highest 

                                       feat2  dt2  rf2  et2  pca2      gbc2
0                             ['total_pkts']  0.0  0.0  0.0   0.0  8.072446
1                ['total_pkts', 'number_ms']  0.0  0.0  0.0   0.0  0.527684
2                              ['number_ms']  0.0  0.0  0.0

In [18]:
main(targets[3])

tba
all combinations generated
all combinations generated
Testing on unseen data: 


Loss Performance sorted from lowest to highest 

                             feat          dt          rf          et  \
5    ['longest_seq', 'max_bytes']  16516000.0  19570960.0  19095070.0   
6                   ['max_bytes']  27316000.0  27316000.0  22892660.0   
1  ['longest_seq', 'total_bytes']  52081250.0  51522540.0  36626390.0   
2                 ['longest_seq']  57246000.0  57246000.0  53172000.0   
3    ['max_bytes', 'total_bytes']  58142250.0  52761522.5  50753125.0   

          pca           gbc  
5  52925000.0  1.968377e+07  
6  27316000.0  2.734223e+07  
1  60946000.0  4.943433e+07  
2  57246000.0  5.729378e+07  
3  61142250.0  4.868972e+07   



Latency Performance sorted from lowest to highest 

                                       feat2      dt2      rf2      et2  \
5                ['number_ms', 'byte_ratio']   4000.0   5737.5   5010.5   
2                              ['number_m

MAKING pipeline: , tst on data with change in loss later

* https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
* https://stackoverflow.com/questions/58781601/parameter-tuning-using-gridsearchcv-for-gradientboosting-classifier-in-python
    
    comb[some index]
    
    dt() - untrained
    
    dt( comb) - train, or grid search on it
    
    et() - 
    
    plot them on top of eachother
    
    test(single dataset - aggregated over subset )# overserved or not
    
    plot the bytes, loss records (loss log), and expanded model predictions

In [ ]:
path = os.path.join(os.getcwd() , "outputs")

comb1 = getAllCombinations(1)
comb2 = getAllCombinations(2)

fname = 'labeled-from-data2_100-12000-true-100-12000.csv'
genviz('seen', fname, 8)
newfname = 's_transform_200-1400__200-1000.csv'
s_t = pd.read_csv(os.path.join(os.getcwd() , 'outputs', newfname))

bob = pd.read_csv(os.path.join(os.getcwd() , "tempdata2", fname))
bob['Seconds'] = bob['Time'] - min(bob['Time'])

bob2 = pd.read_csv(os.path.join(os.getcwd() , 'outputs', newfname))

X, y = s_t[comb1[4]], s_t['loss']

# combined csv
X_all = pd.read_csv(os.path.join(os.getcwd() , 'outputs', 'combined_transform.csv'))
y_all = X_all['loss']

# traintestsplit
X_train, X_test, y_train, y_test = train_test_split(X_all[comb1[4]], y_all,
                                                   random_state=0)

#### new plot (overlay bytes and preds)

In [ ]:
df_small = pd.DataFrame(p_2).reset_index()
df_small['index'] = df_small['index'].apply(lambda x: list(range(0, 17)))
#print(df_small)
df_small = df_small.explode('index').reset_index().rename(columns={"index": "Seconds", 0: "loss"})
overlay = pd.DataFrame(df_small[['loss', 'Seconds']])
overlay_17 = overlay[17:]



In [ ]:
overlay_17.dtypes

In [ ]:
bobble = pd.merge( overlay_17, bob, on='Seconds')

In [ ]:
from matplotlib.lines import Line2D     
def plot_detailed_bytes(df, col='1->2Bytes', rollsec=10, labelcol ='loss_x'):
    rollcolor = '#6c2b6d'
    detailcolor = '#e98d6b'
    
    ax = plt.figure(figsize=(18,8))
    df[col].plot(title=f'{col}/s Rate', color=detailcolor)
    df[col].rolling(rollsec).mean().bfill().plot(color=rollcolor)
    plt.axvline(x=180, color='r')
    
    for i in df[df['event'] == 'drop'].index:
        
        plt.axvline(x=i, color='y', alpha=.45)
    custom_lines = [Line2D([0], [0], color=detailcolor, lw=2),
        Line2D([0], [0], color=rollcolor, lw=2),
        Line2D([0], [0], color='y', lw=2, alpha=0.45),
        Line2D([0], [0], color='r', lw=2)]
    plt.legend(custom_lines, 
               [f'{col} per Second', f'{col} per Second ({rollsec}s rolling avg)', 'Packet drop', '180s Mark'], 
               loc='upper right', framealpha=1);

In [ ]:
bobble.head()

In [ ]:
import matplotlib.lines
plot_detailed_bytes(bobble)

In [ ]:
#clf = GridSearchCV(GradientBoostingClassifier(), parameters,scoring=scoring,refit=False,cv=2, n_jobs=-1)